<a href="https://colab.research.google.com/github/AnishaA-git/approximate_nearest_algorithm_DM/blob/master/Approximate_Nearest_Neighbor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment: Approximate Nearest Neighbor

Steps to follow and complete this assignment:

1. Select a dataset which has train, test data, item_features and item_feature_labels.
2. Or create from "lightfm" library.
3. Train the data and test the data using LightFM Model.
4. Save the Model Item_embeddings into vector.
5. Pickle the data.
6. Load the data.
7. Use different Algorithms as mentioned in the assignment.


Reference: https://making.lyst.com/lightfm/docs/datasets.html

In [1]:
!pip install lightfm

In [2]:
from lightfm import LightFM
from lightfm.datasets import fetch_stackexchange
import pickle

# Import Data of StackExchange

Fetch a dataset from the **StackExchange network**.

The datasets contain users answering questions: an interaction is defined as a user answering a given question.

The following datasets from the StackExchange network are available:

1. **CrossValidated**: From stats.stackexchange.com. Approximately 9000 users, 72000 questions, and 70000 answers.
2. **StackOverflow**: From stackoverflow.stackexchange.com. Approximately 1.3M users, 11M questions, and 18M answers.

I tried using both but the StackOverflow dataset is huge and thus takes a lot of time for further computation and processing.

In [3]:
stackexchange = fetch_stackexchange('crossvalidated')
train = stackexchange['train']
test = stackexchange['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features = stackexchange['item_features'], epochs=20 )

item_vectors = stackexchange['item_features'] * model.item_embeddings

# Dump and create Pickle file of the Dataset

In [4]:
with open('stackexchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange['item_feature_labels'], "vector": item_vectors}, f)

In [5]:
def load_data():
    with open('stackexchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:72357', 'question_id:72358', 'question_id:72359'],
       dtype='<U17'),
 'vector': array([[-5.3533729e-02, -5.6464355e-02, -8.1374869e-02, ...,
         -9.6894421e-02,  5.7313439e-02, -1.6908595e-02],
        [-6.6331118e-02, -6.5165102e-02, -5.5008505e-02, ...,
         -1.6171990e-01,  5.1249313e-05, -4.6256021e-02],
        [ 1.4410098e-02,  5.5407797e-04, -7.0389472e-02, ...,
          6.3761450e-02,  6.2974453e-02,  3.7036315e-02],
        ...,
        [ 9.7188018e-03, -4.5543662e-03,  1.9512522e-03, ...,
         -1.0468926e-02,  7.3954579e-03,  9.1926325e-03],
        [ 1.7524811e-02,  9.5676677e-03, -1.6021287e-02, ...,
         -1.5368313e-02,  2.6251541e-02,  8.5461931e-03],
        [ 2.9871691e-02, -1.2285559e-01, -1.0679848e-02, ...,
         -8.9717414e-03, -2.3016008e-02,  2.9397048e-02]], dtype=float32)}

In [6]:
!pip install faiss-gpu
import faiss

# LSH: Locality Sensitive Hashing

In [7]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [8]:
index = LSHIndex(data["vector"], data["name"])
index.build()

In [9]:
stack_vector, stack_name = data['vector'][90:91], data['name'][90]
simlar_stack_questions = '\n* '.join(index.query(stack_vector))
print(f"The most similar staxk to {stack_name} are:\n* {simlar_stack_questions}")

The most similar staxk to question_id:90 are:
* question_id:320
* question_id:90
* question_id:607
* question_id:243
* question_id:608
* question_id:938
* question_id:1015
* question_id:1024
* question_id:304
* question_id:172


# Exhaustive Search using Faiss

In [10]:
class ExhaustiveIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [11]:
index = ExhaustiveIndex(data["vector"], data["name"])
index.build()

In [12]:
stack_vector, stack_name = data['vector'][90:91], data['name'][90]
simlar_stack_questions = '\n* '.join(index.query(stack_vector))
print(f"The most similar staxk to {stack_name} are:\n* {simlar_stack_questions}")

The most similar staxk to question_id:90 are:
* question_id:90
* question_id:88
* question_id:159
* question_id:9159
* question_id:14515
* question_id:3
* question_id:3752
* question_id:5197
* question_id:16477
* question_id:6280


# Vector Encoding using Product Quantization

In [13]:
class ProductIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [14]:
index = ProductIndex(data["vector"], data["name"])
index.build()

In [15]:
stack_index = 90
stack_vector = data['vector'][stack_index:stack_index+1]
print(f"The most simillar stack to {data['name'][stack_index]} are:")
index.query(stack_vector)

The most simillar stack to question_id:90 are:


['question_id:90',
 'question_id:63',
 'question_id:3579',
 'question_id:56',
 'question_id:328',
 'question_id:19',
 'question_id:5031',
 'question_id:1292',
 'question_id:485',
 'question_id:381']

In [16]:
!pip install annoy
import annoy

# Trees and Graphs using Annoy

In [17]:
class TreesIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [18]:
index = TreesIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [19]:
stack_vector, stack_name = data['vector'][90], data['name'][90]
simlar_stack_questions = '\n* '.join(index.query(stack_vector))
print(f"The most similar stack to {stack_name} are:\n* {simlar_stack_questions}")

The most similar stack to question_id:90 are:
* question_id:90
* question_id:88
* question_id:36
* question_id:82
* question_id:3579
* question_id:66426
* question_id:48
* question_id:57585
* question_id:96
* question_id:370


In [20]:
!pip install nmslib
import nmslib

# HNSW: Hierarchical Navigable Small World Algorithm

In [21]:
class HNSWIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [22]:
index = HNSWIndex(data["vector"], data["name"])
index.build()

In [23]:
stack_vector, stack_name = data['vector'][90], data['name'][90]
simlar_stack_questions = '\n* '.join(index.query(stack_vector))
print(f"The most similar stack to {stack_name} are:\n* {simlar_stack_questions}")

The most similar stack to question_id:90 are:
* question_id:90
* question_id:88
* question_id:67247
* question_id:3
* question_id:36
* question_id:159
* question_id:121
* question_id:82
* question_id:86
* question_id:11
